### 5.3.3 Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Convert your numpy arrays to PyTorch tensors
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).long()
X_valid, y_valid = torch.tensor(X_valid).float(), torch.tensor(y_val).long()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).long()

# Convert your numpy arrays to PyTorch tensors
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).long()
X_valid, y_valid = torch.tensor(X_valid).float(), torch.tensor(y_valid).long()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).long()

/var/folders/bp/kpcs2z7s5_54t0p548zr3tb00000gn/T/ipykernel_55909/4281515862.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).long()
/var/folders/bp/kpcs2z7s5_54t0p548zr3tb00000gn/T/ipykernel_55909/4281515862.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_valid, y_valid = torch.tensor(X_valid).float(), torch.tensor(y_val).long()
/var/folders/bp/kpcs2z7s5_54t0p548zr3tb00000gn/T/ipykernel_55909/4281515862.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_te

In [ ]:
# Define your neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to train the model
def train_model(model, criterion, optimizer, train_loader):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass
        loss.backward()
        optimizer.step()

# Function to evaluate the model
def evaluate_model(model, criterion, valid_loader):
    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in valid_loader:
            val_outputs = model(X_batch)
            val_loss = criterion(val_outputs, y_batch)
            val_preds = torch.argmax(val_outputs, dim=1)
            accuracy = accuracy_score(y_batch.numpy(), val_preds.numpy())

    return val_loss.item(), accuracy

# Set up hyperparameter search space
param_grid = {
    'hidden_size': [100, 200, 300],
    'learning_rate': [0.001, 0.01, 0.1],
    'weight_decay': [0.0001, 0.001, 0.01],
}

# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the best model and accuracy
best_model = None
best_accuracy = 0.0

# Perform hyperparameter search
for params in ParameterGrid(param_grid):
    model = SimpleNN(input_size=X_train.shape[1], hidden_size=params['hidden_size'], output_size=len(np.unique(y_train)))
    criterion = torch.Variable(nn.CrossEntropyLoss, requires_grad=True)
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

    for train_idx, valid_idx in cv.split(X_train):
        train_dataset = TensorDataset(X_train[train_idx], y_train[train_idx])
        valid_dataset = TensorDataset(X_train[valid_idx], y_train[valid_idx])

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=len(valid_idx), shuffle=False)

        for epoch in range(100):  # Set the number of epochs to 100
            train_model(model, criterion, optimizer, train_loader)

        val_loss, accuracy = evaluate_model(model, criterion, valid_loader)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

# Report validation score of the best model
print('Best validation accuracy:', best_accuracy)

AttributeError: module 'torch' has no attribute 'Variable'